<a href="https://colab.research.google.com/github/soichi-fujiwara/jupyter-notebook/blob/master/Web_scraping_Accidents.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install beautifulsoup4
!pip install mecab-python3

In [0]:
import sys
from datetime import *
from time import *

def get_yobi(inDate):
  
  yobi = ["月","火","水","木","金","土","日"]

  a = datetime.strptime(inDate,'%Y/%m/%d')

  #return yobi[a.weekday()]
  return a.weekday()

In [0]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
import pandas as pd
#import MeCab
from google.colab import files

#https://accident.laboneko.jp/accidents?limit=10&offset=0
#https://accident.laboneko.jp/accidents?limit=11680&offset=0

# 親ページURL
url_pr = 'https://accident.laboneko.jp/accidents?limit=11680&offset=0'

# データフレームを定義
columns = ["date","time","yobi","line","from_st","to_st","st"]
df = pd.DataFrame(columns=columns)

html_doc = urlopen(url_pr)

# BeautifulSoupでスクレイピング
soup = BeautifulSoup(html_doc, 'html.parser')
# ラッピング単位のクラスを取得
table  = soup.find_all("table", {"class": "table table-striped"})[0]
rows = table.findAll("tr")

for row in rows:
  dataRow = []
  for cell in row.findAll("td"):
    dataRow.append(cell.text)

  #listに何か入っている場合
  if dataRow:

    v_date = dataRow[0][0:10]
    v_time = dataRow[0][11:16]
    v_yobi = get_yobi(v_date)

    v_line = dataRow[1]

    v_st_info = dataRow[2]
    v_st_info_sp = v_st_info.split("〜")

    if len(v_st_info_sp) >= 2:
      v_from_st = v_st_info_sp[0]
      v_to_st = v_st_info_sp[1]
      v_st = v_st_info_sp[0]
    else:
      v_from_st = v_st_info
      v_to_st = v_st_info
      v_st = v_st_info

    # 各データをデータフレームに格納
    se = pd.Series([v_date,v_time,v_yobi,v_line,v_from_st,v_to_st,v_st], columns)
    df = df.append(se, columns)

# 収集したデータをCSV形式で保存
filename = "Accident_list.csv"
df.to_csv(filename, encoding = 'utf-8')
#files.download(filename)


##ファイルがあればここから実行

In [23]:
import pandas as pd

#index_col=0を付けないとUnnamedというカラムが作成されてしまう。
df_ac = pd.read_csv("Accident_list.csv")

#不要な項目は削除
df_ac["line"].value_counts().head(30)

東武東上線           291
山陽本線_(JR西日本)    263
京浜東北線           245
中央快速線           244
東海道線_(JR東海)     244
JR神戸線           235
常磐線             233
鹿児島本線           223
小田急小田原線         216
西武新宿線           187
宇都宮線            182
京王線             180
京急本線            172
東海道線_(JR西日本)    157
西武池袋線           153
東武伊勢崎線          152
高崎線             151
東海道線_(JR東日本)    150
予讃線             146
日豊本線            145
中央・総武緩行線        142
阪急京都本線          140
名鉄名古屋本線         136
東急田園都市線         131
山手線             125
東北本線            122
近鉄大阪線           118
京阪本線            112
北陸本線            109
京成本線            109
Name: line, dtype: int64

In [24]:
import itertools

kanto_line = pd.read_table('kanto_line.txt', header=None)

#dataframeをlist化
kanto_line_list = kanto_line.values.tolist()
#2次元listから1次元listへ
kanto_line_list = list(itertools.chain.from_iterable(kanto_line_list))

print(kanto_line_list)

['東武東上線', '京浜東北線', '東海道線_(JR東海)', '中央快速線', '常磐線', '小田急小田原線', '西武新宿線', '宇都宮線', '京王線', '京急本線', '西武池袋線', '東武伊勢崎線', '高崎線', '東海道線_(JR東日本)', '東急田園都市線', '山手線', '東北本線', '京成本線', '横浜線', '総武快速線', '横須賀線', '南武線', '埼京線', '相鉄本線', '東武野田線']


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: read_table is deprecated, use read_csv instead, passing sep='\t'.
  This is separate from the ipykernel package so we can avoid doing imports until


In [28]:
#関東(首都圏)に影響を与えそうな路線のデータのみ抽出
new_data = df_ac[df_ac['line'].isin(kanto_line_list)]
new_data = new_data.reset_index()

#不要な項目を削除
new_data = new_data.drop("idx",axis=1)
new_data = new_data.drop("index",axis=1)

new_data.head(10)

,date,time,yobi,line,from_st,to_st,st
0,2019/08/07,18:00,2,宇都宮線,東大宮駅,蓮田駅,東大宮駅
1,2019/08/03,23:32,5,小田急小田原線,新松田駅,開成駅,新松田駅
2,2019/08/01,15:23,3,総武快速線,稲毛駅,稲毛駅,稲毛駅
3,2019/07/31,18:09,2,宇都宮線,赤羽駅,赤羽駅,赤羽駅
4,2019/07/31,09:52,2,中央快速線,武蔵小金井駅,武蔵小金井駅,武蔵小金井駅
5,2019/07/30,07:02,1,小田急小田原線,生田駅,生田駅,生田駅
6,2019/07/28,15:03,6,西武池袋線,入間市駅,仏子駅,入間市駅
7,2019/07/27,12:49,5,東武東上線,高坂駅,高坂駅,高坂駅
8,2019/07/25,16:06,3,小田急小田原線,相武台前駅,相武台前駅,相武台前駅
9,2019/07/22,21:19,0,常磐線,柏駅,柏駅,柏駅


In [0]:
!pip install pandas-profiling

In [29]:
import pandas_profiling as pdp
from IPython.display import HTML

profile = pdp.ProfileReport(new_data)
profile.to_file(outputfile="myOutputFile_line.html")

#html化して表示
HTML(filename='myOutputFile_line.html')

Number of variables,7
Number of observations,3926
Total Missing (%),0.0%
Total size in memory,214.8 KiB
Average record size in memory,56.0 B
Numeric,1
Categorical,6
Boolean,0
Date,0
Text (Unique),0
Rejected,0


# **profilingで分かったこと**
・曜日は関係無さそう

・分析する上で情報が不足している

・正規化、細分化できる情報は無いか

In [46]:
# データフレームを定義
#通勤時間帯/帰宅時間帯/日中/それ以外で分けてみよう

#コピーを付けないとコピー元も変更されてしまう
new_df = new_data.copy()

#カラム「time」をコピーして追加
#(後でカテゴリ分けしたい為)
new_df = new_df.assign(time_category=new_df['time'])
new_df = new_df.assign(season=new_df['date'])

#-------------------------------------------------------------------------------------------
#時間帯
#-------------------------------------------------------------------------------------------
#数値型で比較したい場合は「.str[0:2].astype(int)」を使用する
new_df.loc[(new_df['time_category'].str[0:2].isin(["04","05"])) ,"time_category"] = "morning_time"
new_df.loc[(new_df['time_category'].str[0:2].isin(["06","07","08","09"])) ,"time_category"] = "comp_time"
new_df.loc[(new_df['time_category'].str[0:2].isin(["17","18","19","20","21"])) ,"time_category"] = "home_time"
new_df.loc[(new_df['time_category'].str[0:2].isin(["22","23","00","01"])) ,"time_category"] = "night_time"
new_df.loc[(new_df['time_category'].str[0:2].isin(["02","03","10","11","12","13","14","15","16"])) ,"time_category"] = "etc_time"

dmy_df = pd.get_dummies(new_df['time_category'])
add_df = pd.merge(new_df, dmy_df, left_index=True, right_index=True)
add_df = add_df.drop("time_category",axis=1)

#-------------------------------------------------------------------------------------------
#気候
#-------------------------------------------------------------------------------------------
add_df.loc[(add_df['season'].str[5:7].isin(["04","05","10","11"])) ,"season"] = "warm"
add_df.loc[(add_df['season'].str[5:7].isin(["06","07","08","09"])) ,"season"] = "hot"
add_df.loc[(add_df['season'].str[5:7].isin(["12","01","02","03"])) ,"season"] = "cold"

dmy_df = pd.get_dummies(add_df['season'])
add_df = pd.merge(add_df, dmy_df, left_index=True, right_index=True)
add_df = add_df.drop("season",axis=1)

#-------------------------------------------------------------------------------------------
#曜日
#-------------------------------------------------------------------------------------------
dmy_df = pd.get_dummies(add_df['yobi'],prefix='yobi')
add_df = pd.merge(add_df, dmy_df, left_index=True, right_index=True)
add_df = add_df.drop("yobi",axis=1)

add_df.head(5)

,date,time,line,from_st,to_st,st,comp_time,etc_time,home_time,morning_time,night_time,cold,hot,warm,yobi_0,yobi_1,yobi_2,yobi_3,yobi_4,yobi_5,yobi_6
0,2019/08/07,18:00,宇都宮線,東大宮駅,蓮田駅,東大宮駅,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0
1,2019/08/03,23:32,小田急小田原線,新松田駅,開成駅,新松田駅,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0
2,2019/08/01,15:23,総武快速線,稲毛駅,稲毛駅,稲毛駅,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0
3,2019/07/31,18:09,宇都宮線,赤羽駅,赤羽駅,赤羽駅,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0
4,2019/07/31,09:52,中央快速線,武蔵小金井駅,武蔵小金井駅,武蔵小金井駅,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0


In [52]:
from bs4 import BeautifulSoup
from urllib.request import urlopen

# 親ページURL
url_pr = 'https://www.jreast.co.jp/passenger/index.html'

# データフレームを定義
columns = ["stname","ninzu"]
df_ppl = pd.DataFrame(columns=columns)

html_doc = urlopen(url_pr)

# BeautifulSoupでスクレイピング
soup = BeautifulSoup(html_doc, 'html.parser')
# ラッピング単位のクラスを取得1
table  = soup.find_all("table", {"class": "passengerTable fl"})[0]
rows = table.findAll("tr")

for row in rows:
  dataRow = []
  for cell in row.findAll("td"):
    dataRow.append(cell.text)

  #listに何か入っている場合
  if dataRow:

    v_stname = dataRow[1]+"駅"
    v_ppl = dataRow[4].replace(",","")

    # 各データをデータフレームに格納
    se = pd.Series([v_stname,v_ppl], columns)
    df_ppl = df_ppl.append(se, columns)

# ラッピング単位のクラスを取得2
table  = soup.find_all("table", {"class": "passengerTable fr"})[0]
rows = table.findAll("tr")

for row in rows:
  dataRow = []
  for cell in row.findAll("td"):
    dataRow.append(cell.text)

  #listに何か入っている場合
  if dataRow:

    v_stname = dataRow[1]+"駅"
    v_ppl = dataRow[4].replace(",","")

    # 各データをデータフレームに格納
    se = pd.Series([v_stname,v_ppl], columns)
    df_ppl = df_ppl.append(se, columns)

#TOP100以下
# 親ページURL
url_pr = 'https://www.jreast.co.jp/passenger/2018_01.html'
html_doc = urlopen(url_pr)

# BeautifulSoupでスクレイピング
soup = BeautifulSoup(html_doc, 'html.parser')
# ラッピング単位のクラスを取得1
table  = soup.find_all("table", {"class": "passengerTable fl"})[0]
rows = table.findAll("tr")

for row in rows:
  dataRow = []
  for cell in row.findAll("td"):
    dataRow.append(cell.text)

  #listに何か入っている場合
  if dataRow:

    v_stname = dataRow[0]+"駅"
    v_ppl = dataRow[3].replace(",","")

    # 各データをデータフレームに格納
    se = pd.Series([v_stname,v_ppl], columns)
    df_ppl = df_ppl.append(se, columns)

# ラッピング単位のクラスを取得2
table  = soup.find_all("table", {"class": "passengerTable fr"})[0]
rows = table.findAll("tr")

for row in rows:
  dataRow = []
  for cell in row.findAll("td"):
    dataRow.append(cell.text)

  #listに何か入っている場合
  if dataRow:

    v_stname = dataRow[0]+"駅"
    v_ppl = dataRow[3].replace(",","")

    # 各データをデータフレームに格納
    se = pd.Series([v_stname,v_ppl], columns)
    df_ppl = df_ppl.append(se, columns)

#    
# 収集したデータをCSV形式で保存
filename = "Train_People.csv"
df_ppl.to_csv(filename, encoding = 'utf-8')

df_ppl

,stname,ninzu
0,新宿駅,789366
1,池袋駅,566994
2,東京駅,467165
3,横浜駅,423651
4,品川駅,383442
5,渋谷駅,370856
6,新橋駅,281971
7,大宮駅,258108
8,秋葉原駅,252267
9,北千住駅,220903


In [44]:
profile2 = pdp.ProfileReport(add_df)
profile2.to_file(outputfile="myOutputFile_line2.html")

#html化して表示
HTML(filename='myOutputFile_line2.html')

Number of variables,21
Number of observations,3926
Total Missing (%),0.0%
Total size in memory,241.6 KiB
Average record size in memory,63.0 B
Numeric,0
Categorical,6
Boolean,15
Date,0
Text (Unique),0
Rejected,0
